In [1]:
!nvidia-smi

Sat Apr  9 16:29:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     8W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers sentencepiece

     |████████████████████████████████| 4.0 MB 33.2 MB/s 
     |████████████████████████████████| 1.2 MB 58.5 MB/s 
     |████████████████████████████████| 77 kB 7.9 MB/s 
     |████████████████████████████████| 6.5 MB 17.4 MB/s 
     |████████████████████████████████| 895 kB 54.7 MB/s 
     |████████████████████████████████| 596 kB 48.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import pipeline

PARAPHRASE_MODEL = 'tuner007/pegasus_paraphrase'

# device = 0 to use GPU 
paraphrase_pipeline = pipeline("text2text-generation", model=PARAPHRASE_MODEL, device=0)

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [4]:
def paraphrase(text):
    # generate 4 alternatives, skip paraphrases that are not questions (= don't end with ?)
    return [p['generated_text'] for p in paraphrase_pipeline(text, num_return_sequences=4) if p['generated_text'].endswith('?')]

In [5]:
from google.colab import files

uploaded = files.upload()

filename = list(uploaded.keys())[0]

Saving try4.6-paraphrased-input-no-clean (1).csv to try4.6-paraphrased-input-no-clean (1).csv


In [6]:
import csv

alternatives = []

with open(filename) as f:
    reader = csv.DictReader(f)

    for line in reader:
        question = line['question'].strip()

        if(not question.endswith('?')):
            # skipping irrelevant questions
            continue

        

        # include original question
        alternatives.append({
                'question': question,
                'answer': line['answer'],
                'context': line['context'],
                'start_pos': line['start_pos']
            })
        
        alts = paraphrase(question)

        # and up to 4 alternatives
        for alt in alts:
            alternatives.append({
                'question': alt,
                'answer': line['answer'],
                'context': line['context'],
                'start_pos': line['start_pos']
            })

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [7]:
with open('extended.csv', 'w') as f:
    writer = csv.DictWriter(f, ['question', 'answer', 'context', 'start_pos'])

    writer.writeheader()
    for line in alternatives:
        writer.writerow(line)

print(len(alternatives))

1348
